# Arquitecturas avanzadas de NLP. Sequence 2 Sequence

[Uno de los mejores resources](https://distill.pub/2016/augmented-rnns/)

Hoy/ahora, veremos una de las arquitecturas más usadas, para un tipo de problema muy concreto, pero muy extendido. Si os acordais de Language Modeling, generábamos secuencias, con una palabra o carácter de semilla, pero podía ir a cualquier parte la secuencia. Con Sequence 2 Sequence (Seq2Seq - S2S), lo que queremos es condicionar el output de nuestra arquitectura a todo un contexto.

![Seq2Seq](https://cdn-images-1.medium.com/max/1600/1*_6-EVV3RJXD5KDjdnxztzg@2x.png)

Esta arquitectura apareció por primera vez en 2014, [paper](https://www.aclweb.org/anthology/D14-1179), y aunque tuvo mucha repercusión, no fue hasta que en 2015... lo vemos despues!

Estas arquitecturas, ya en 2014 mostraron gran potencial, y se confirmó cuando empresas como Google, decidieron cambiar productos tan importantes como el Google Translate a arquitecturas similares a esta. Os dejó un blog [post](https://codesachin.wordpress.com/2017/01/18/understanding-the-new-google-translate/) para que veais, y el [paper](https://arxiv.org/pdf/1609.08144.pdf) original dónde se explica el funcionamiento de la arquitectura. Otra aplicación bastante interesante, es la de generar respuestas a correos electrónicos de forma automática [info aquí](https://ai.googleblog.com/2015/11/computer-respond-to-this-email.html).

### Vale bien, pero que hace esto?

El dibujo de arriba, si seguis el timeline (time-steps), queda bastante claro. Primero codificamos un mensaje (steps-1-4), luego inicializamos el decoder (flecha entre 4 y 5), y empezamos la decodificación (5-7). 

Durante el entreno, se pasan dos inputs distintos, el input del encoder, y el del decoder. El input del encoder aquí sería




In [0]:
input_encoder = ["<SOS>", "how", "are", "you", "?", "<EOS>"]
input_decoder = ["<SOS>", "I", "am", "good", "<EOS>"]
target_decoder = input_decoder[1:]
target_decoder

['I', 'am', 'good', '<EOS>']

Fijaros que el encoder no tiene un target, sino que calcularemos la loss en el decoder, y pasaremos el gradiente hacía atrás desde el decoder, hasta el primer step del encoder.

Del dibujo de arriba, ya lo habeis visto absolutamente todo. Las celdas azules verdes y azules son...? Normalmente son celdas RNN, ya sean vanilla RNN, LSTMs o GRUs (que aunque no las hemos introducido en el curso, son una variamente algo más "barata" que las LSTMs).

La única flecha que os debería preocupar es la de verde a azul. En esa parte estamos copiando el último estado de nuestra encoder-RNN, y usandolo como estado inicial en el decoder-RNN. Si os acordáis del dibujo, de las LSTMs, básicamente queremos h_t, cuando t = T, es decir en su último estado.

![](https://i.imgur.com/nE53oh1.png)

### Keras tip

![](https://i.imgur.com/V9Pkm8d.png)

Keras nos permite obtener el estado de una LSTM, seteando return_state a True.

Esto modifica el output, devolviendonos 3 tensores, el output, el hidden state, i el cell state. En caso normal, el output i el hidden state, son lo mismo, pero esto puede ser modificado con el keyword return_sequences, que nos permite obtener una matriz con un output por timestep, el hidden state (del último estado) y el cell_state del último estado.

Ahora que ya sabemos como recoger el hidden state, ya podemos ir a implementar un Seq2Seq en Keras.

No implementaremos nada en numpy, porque ya tenemos casi todo implementado (almenos el forward pass) en sesiones pasadas, con lo que saltaremos a Keras directamente.


## Imports

In [1]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.layers import TimeDistributed
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import numpy as np
from random import shuffle, choice, sample

import pprint as pp

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display

sns.set(color_codes=True)

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

Using TensorFlow backend.


## Data

In [2]:
def generate_dummy_data():
    x = [ix for ix in range(100)]
    data = []
    for ix_source in range(3, 5):
        for ix_target in range(3, 5):
            for ix, _ in enumerate(x):
                data.append((x[ix:ix+ix_source], x[ix+ix_source:ix+ix_target*2]))
    return data
    

In [3]:
dummy_data = generate_dummy_data()

In [4]:
shuffle(dummy_data)

In [6]:
dummy_data[0]

([47, 48, 49, 50], [51, 52, 53, 54])

## Data Preprocess

In [7]:
data_tr = []

for i, (inp, out) in enumerate(dummy_data):
    inp.insert(0, '<SOS>')
    out.insert(0, '<SOS>')
    
    inp.append('<EOS>')
    out.append('<EOS>')
    
    data_tr.append((inp, out))

In [8]:
maxlen = max([len(x) for x, _ in dummy_data])
maxlen

6

In [9]:
#vocabulary preparation
vocab = []
for inp, out in data_tr:
    vocab+=[w for w in inp]
    vocab+=[w for w in out]
vocab = list(set(vocab))
vocab.insert(0, '<PAD>')
vocab.append('<UNK>')
print(vocab)

['<PAD>', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, '<EOS>', '<SOS>', '<UNK>']


In [10]:
w2id = {w:i for i, w in enumerate(vocab)}
id2w = {w:i for i, w in w2id.items()}

In [44]:
data_train = []

for inp, out in data_tr:
    enc_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in inp]
    dec_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in out]
    dec_out = [w2id[w] if w in w2id else w2id['<UNK>'] for w in out[1:]]
    
    data_train.append((enc_in, dec_in, dec_out))

In [45]:
data_train[0]

([102, 48, 49, 50, 51, 101], [102, 52, 53, 54, 55, 101], [52, 53, 54, 55, 101])

In [46]:
data_tr[0]

(['<SOS>', 47, 48, 49, 50, '<EOS>'], ['<SOS>', 51, 52, 53, 54, '<EOS>'])

## Auxiliary Functions

In [55]:
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 1 == 0  and epoch>0:
            nb_samples = 1
            data_t = sample(data_tr, nb_samples)
            data_test = []
            for inp, out in data_t:
                ind_enc_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in inp]
                ind_dec_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in out]
                data_test.append((ind_enc_in,ind_dec_in))

            params = {
                'max_encoder_len': maxlen + 2,
                'max_decoder_len': maxlen + 2,
                'target_len': len(vocab)
                }

            encoder_input_data = np.zeros(shape=(nb_samples, params['max_encoder_len']))    
            decoder_input_data = np.zeros(shape=(nb_samples, params['max_decoder_len']))

            for i, (ei, di) in enumerate(data_test):
                for j, idx in enumerate(ei):
                    encoder_input_data[i, j] = idx
                for j, idx_di in enumerate(di):
                    decoder_input_data[i, j] = idx_di

            result = self.model.predict([encoder_input_data, decoder_input_data])
            for r, original in zip(result, data_t):
                original_sentence = original[0]
                idx = np.argmax(r, axis=1)
                print(idx)
                repr_out = []
                for ix in idx:
                    token = id2w[ix]
                    if token == '<EOS>':
                        break
                    else:
                        repr_out.append(token)
                #print('   '*40, end='\r')
                print('Test Sample epoch({}): {} ====> {}'.format(epoch, original_sentence, " ".join(repr_out)))

In [0]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % 3 == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g--', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b--', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

## Architecture definition

In [56]:
class Seq2Seq:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
    
    def compile_basic_seq2seq(self, params={}):
        
        encoder_inputs = Input(shape=(params['max_encoder_len'], ), name='encoder_input')
        embedding_layer = Embedding(input_dim=params['vocab'], output_dim=params['emb_feats'], name='embedding_layer')
        
        encoder_embedding = embedding_layer(encoder_inputs)
        
        encoder = LSTM(params['hidden_size'], return_state=True, name='encoder')
        
        encoder_outputs, state_h, state_c = encoder(encoder_embedding)
        print(encoder_outputs.shape, state_h.shape, state_c.shape)
        # inicializar el decoder con el ultimo estado de nuestro encoder
        encoder_states = [state_h, state_c]
        
        decoder_inputs = Input(shape=(params['max_decoder_len'], ), name='decoder_input')
        embedding_layer_dec = Embedding(input_dim=params['vocab'], output_dim=params['emb_feats'], name='embedding_layer_decoder')
        
        decoder_embedding = embedding_layer_dec(decoder_inputs)
        decoder = LSTM(params['hidden_size'], return_state=True, return_sequences=True, name='decoder')
        
        decoder_out, _, _ = decoder(decoder_embedding, initial_state=encoder_states)
        
        decoder_dense = Dense(params['target_size'], activation='softmax', name='softmax')
        
        decoder_outputs = TimeDistributed(decoder_dense)(decoder_out)
        
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
        
        model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()
        
        return model
        
    def train(self, model, data, params={}):
        
        callbacks = self._get_callbacks()
        if 'shuffle' in params and params['shuffle']:
            shuffle(data)
        
        encoder_input_data = np.zeros(shape=(len(data), train_params['max_encoder_len']))    
        decoder_input_data = np.zeros(shape=(len(data), train_params['max_decoder_len']))
        decoder_target_data = np.zeros(shape=(len(data), train_params['max_decoder_len'], train_params['target_len']))
        for i, (ei, di,dt) in enumerate(data):            
            for j, idx in enumerate(ei):
                encoder_input_data[i, j] = idx
            for j, idx_di in enumerate(di):
                decoder_input_data[i, j] = idx_di
            for j, idx_dt in enumerate(dt):
                decoder_target_data[i, j, idx_dt] = 1       
                
        model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)
            
    def predict(self, model, data, params={}):
        
        nb_samples = len(data)
        data_t = sample(data_tr, nb_samples)
        data_test = []
        predicted_out = []
        for samp in range(nb_samples):
            for inp, out in data_t:
                ind_enc_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in inp]
                ind_dec_in = [w2id[w] if w in w2id else w2id['<UNK>'] for w in out]
                data_test.append((ind_enc_in,ind_dec_in))


            encoder_input_data = np.zeros(shape=(1, params['max_encoder_len']))    
            decoder_input_data = np.zeros(shape=(1, params['max_decoder_len']))

            for i, (ei, di) in enumerate(data_test):
                for j, idx in enumerate(ei):
                    encoder_input_data[i, j] = idx
                for j, idx_di in enumerate(di):
                    decoder_input_data[i, j] = idx_di

            result = self.model.predict([encoder_input_data, decoder_input_data])
            for r, original in zip(result, data_t):
                original_sentence = original[0]
                idx = np.argmax(r, axis=1)
                print(idx)
                repr_out = []
                for ix in idx:
                    token = id2w[ix]
                    if token == '<EOS>':
                        break
                    else:
                        repr_out.append(token)
            predicted_out.append(repr_out)
        return predicted_out
                        
                        
    def load(self, model_path='seq2seq.h5'):
        return load_model(model_path)
    
    def _get_callbacks(self, model_path='seq2seq.h5'):
        es = EarlyStopping(monitor='loss', patience=20, mode='auto', verbose=1)
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only=True, save_weights_only=False, period=2)
        st = Sampletest()
        # hd = HistoryDisplay()
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2,
              patience=5, min_lr=0.0001, verbose=1)
        return [st, rlr]

## Compile model definition

In [24]:
compile_params = {
    'vocab': len(vocab),
    'emb_feats': 50,
    'hidden_size': 256,
    'target_size': len(vocab),
    'input_size': len(vocab),
    'max_encoder_len': maxlen+2,
    'max_decoder_len': maxlen+2    
}

In [57]:
s2s = Seq2Seq()
s2s_model = s2s.compile_basic_seq2seq(params=compile_params)    

(?, 256) (?, 256) (?, 256)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 8)            0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, 8)            0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 8, 50)        5200        encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_layer_decoder (Embedd (None, 8, 50)        5200        decoder_input[0][0]              
__________________________________________________________________________________

## Seq2Seq Train

In [58]:
train_params = {
    'epochs': 1000,
    'batch_size': 32,
    'shuffle': True,
    'target_len': len(vocab),
    'max_encoder_len': maxlen +2,
    'max_decoder_len': maxlen +2
    
}

s2s.train(model=s2s_model, data=data_train, params=train_params)

Epoch 1/1000
400/400 [==============================] - 2s 6ms/step - loss: 2.5118 - acc: 0.0875
Epoch 2/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.5086 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(1): ['<SOS>', 91, 92, 93, '<EOS>'] ====> 
Epoch 3/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.5054 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(2): ['<SOS>', 18, 19, 20, '<EOS>'] ====> 
Epoch 4/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.5021 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(3): ['<SOS>', 7, 8, 9, '<EOS>'] ====> 
Epoch 5/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4989 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(4): ['<SOS>', 33, 34, 35, '<EOS>'] ====> 
Epoch 6/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4957 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample

400/400 [==============================] - 1s 2ms/step - loss: 2.4828 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(9): ['<SOS>', 34, 35, 36, 37, '<EOS>'] ====> 
Epoch 11/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4796 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(10): ['<SOS>', 46, 47, 48, 49, '<EOS>'] ====> 
Epoch 12/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4763 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(11): ['<SOS>', 7, 8, 9, 10, '<EOS>'] ====> 
Epoch 13/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4731 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(12): ['<SOS>', 92, 93, 94, '<EOS>'] ====> 
Epoch 14/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4699 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(13): ['<SOS>', 14, 15, 16, 17, '<EOS>'] ====> 
Epoch 15/1000
400/400 [==============

400/400 [==============================] - 1s 2ms/step - loss: 2.4537 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(18): ['<SOS>', 98, 99, '<EOS>'] ====> 
Epoch 20/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4504 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(19): ['<SOS>', 73, 74, 75, '<EOS>'] ====> 
Epoch 21/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4471 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(20): ['<SOS>', 58, 59, 60, 61, '<EOS>'] ====> 
Epoch 22/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4438 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(21): ['<SOS>', 39, 40, 41, '<EOS>'] ====> 
Epoch 23/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4405 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(22): ['<SOS>', 66, 67, 68, 69, '<EOS>'] ====> 
Epoch 24/1000
400/400 [======================

[101 101 101 101 101 101 101 101]
Test Sample epoch(26): ['<SOS>', 77, 78, 79, 80, '<EOS>'] ====> 
Epoch 28/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4239 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(27): ['<SOS>', 34, 35, 36, 37, '<EOS>'] ====> 
Epoch 29/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4205 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(28): ['<SOS>', 39, 40, 41, 42, '<EOS>'] ====> 
Epoch 30/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4172 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(29): ['<SOS>', 28, 29, 30, 31, '<EOS>'] ====> 
Epoch 31/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4138 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(30): ['<SOS>', 44, 45, 46, 47, '<EOS>'] ====> 
Epoch 32/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.4103 - acc: 0.1250
[101 101 101 10

400/400 [==============================] - 1s 2ms/step - loss: 2.3965 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(35): ['<SOS>', 82, 83, 84, '<EOS>'] ====> 
Epoch 37/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.3930 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(36): ['<SOS>', 31, 32, 33, 34, '<EOS>'] ====> 
Epoch 38/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.3895 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(37): ['<SOS>', 34, 35, 36, 37, '<EOS>'] ====> 
Epoch 39/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.3859 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(38): ['<SOS>', 97, 98, 99, '<EOS>'] ====> 
Epoch 40/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.3824 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(39): ['<SOS>', 34, 35, 36, '<EOS>'] ====> 
Epoch 41/1000
400/400 [==================

...

[101 101 101 101 101 101 101 101]
Test Sample epoch(59): ['<SOS>', 57, 58, 59, '<EOS>'] ====> 
Epoch 61/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.3016 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(60): ['<SOS>', 42, 43, 44, '<EOS>'] ====> 
Epoch 62/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2975 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(61): ['<SOS>', 31, 32, 33, 34, '<EOS>'] ====> 
Epoch 63/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2934 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(62): ['<SOS>', 25, 26, 27, 28, '<EOS>'] ====> 
Epoch 64/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2893 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(63): ['<SOS>', 75, 76, 77, 78, '<EOS>'] ====> 
Epoch 65/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2851 - acc: 0.1250
[101 101 101 101 101 10

400/400 [==============================] - 1s 2ms/step - loss: 2.2685 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(68): ['<SOS>', 85, 86, 87, 88, '<EOS>'] ====> 
Epoch 70/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2643 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(69): ['<SOS>', 5, 6, 7, 8, '<EOS>'] ====> 
Epoch 71/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2602 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(70): ['<SOS>', 44, 45, 46, '<EOS>'] ====> 
Epoch 72/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2561 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(71): ['<SOS>', 74, 75, 76, '<EOS>'] ====> 
Epoch 73/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2520 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(72): ['<SOS>', 68, 69, 70, '<EOS>'] ====> 
Epoch 74/1000
400/400 [======================

[101 101 101 101 101 101 101 101]
Test Sample epoch(76): ['<SOS>', 60, 61, 62, '<EOS>'] ====> 
Epoch 78/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2321 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(77): ['<SOS>', 97, 98, 99, '<EOS>'] ====> 
Epoch 79/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2283 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(78): ['<SOS>', 80, 81, 82, '<EOS>'] ====> 
Epoch 80/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2246 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(79): ['<SOS>', 32, 33, 34, '<EOS>'] ====> 
Epoch 81/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2210 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(80): ['<SOS>', 66, 67, 68, 69, '<EOS>'] ====> 
Epoch 82/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2174 - acc: 0.1250
[101 101 101 101 101 101 101 10

[101 101 101 101 101 101 101 101]
Test Sample epoch(85): ['<SOS>', 79, 80, 81, '<EOS>'] ====> 
Epoch 87/1000
400/400 [==============================] - 1s 2ms/step - loss: 2.2013 - acc: 0.1250
[101 101 101 101 101 101 101 101]
Test Sample epoch(86): ['<SOS>', 65, 66, 67, 68, '<EOS>'] ====> 
Epoch 88/1000
288/400 [====================>.........] - ETA: 0s - loss: 2.1770 - acc: 0.1250

KeyboardInterrupt: ignored

## Predict

In [1]:
predict_params = {
            'max_encoder_len': maxlen + 2,
            'max_decoder_len': maxlen + 2,
            'target_len': len(vocab)
            }

s2s.predict(model=s2s_model, data=['', ''], params=predict_params)

# data tiene que ser una lista de esas predicciones que quereis hacer. Fijaros en la implementación de Sampletest más arriba